A Dive Into the Constrained Portfolio Optimization Program
---
Consider a constrained quadratic program
$$
\max\limits_{\{x_{i}\}_i} \sum_{i} a_{i} x_{i} - b_{i} x_{i}^{2} \\
\text{subject to }
\left\{\begin{array}{l}
        \sum_{i} q_{i} x_{i} = s  \\
%         \text{ and } \\
        x_{i} \geq 0  \text{ for each } i 
    \end{array}\right.
$$

The primal formulation of this problem is
$$
\max_{ \{x_{i}\}_{i}} \min _{v}\left\{\sum_{i} f_{i}(x)+v\left(q^{\prime} x-s\right)\right\}
$$

$$
\text{ where } f_{i}(x) = a_{i} x-b_{i} x^{2}+p_i(x)
$$

$$
\text{and }  p_i(x) = 
\left\{ \begin{array}{ll} \infty & \text{if } x_i < 0 \\ 0 & \text{otherwise }
\end{array} \right.
$$

where $v$ is the Lagrange multiplier on the linear constraint. 

We can instead solve the dual problem:
$$ \min_{v} \underbrace{\max_{ \{x_{i}\}_{i}} \left\{\sum_{i} f_{i}(x)+v\left(q^{\prime} x-s\right)\right\}}_\text{$g(x)$} $$

The First Order Conditions of $g(x)$ are given by
$$
\frac{\nabla_{i} g(x)}{\partial x_{i}}=a_{i}-2 b x_{i}+v q_{i}=0
$$

$$
\text{And so, at the optimum } x_{i}^{*}=\max \left\{\frac{a_{i+} v q_{i}}{2 b_{i}}, 0\right\}.
$$

Substituting $x^{*}$ into the dual objective, we obtain:

$$
\min_{v}\left\{\sum_{i} h_{i}(x_{i}^{*})+v\left(s\right)\right\}
$$

$$\text{where: }
h_{i}(x_{i}^{*}) = 
\left\{ \begin{array}{ll} 
\frac{\left(a_{i}+v q_{i}\right)^{2}}{2 b_{i}}-b_{i}\left(\frac{a_{i}+vq_{i}}{2 b_{i}}\right)^{2} & \text{if } 
\frac{a_{i} + v q_{i}}{2 b_{i}} > 0 \\
0 & \text{otherwise } \end{array} \right.
$$

$$ \text{Simplifying: }
h_{i}(x_{i}^{*}) =
\left\{ \begin{array}{ll}
\frac{1}{4 b_{i}}\left(a_{i}+v q_{i}\right)^{2} & \text{if  } \frac{a_{i} + v q_{i}}{2 b_{i}} > 0 \\
0 & \text{otherwise } \end{array} \right. $$

Thus, the solution to the original problem is the $v_{k}^{*}$ that minimizes Equation $(*)$ with $k$ non-zero components with the form $x_{i}^{*} = \frac{a_{i} + v_{k}^{*} q_{i}}{2 b_{i}}$.

Noting that $\frac{a_{i} + v q_{i}}{2 b_{i}} > 0 \iff v > \frac{-a_{i}}{q_{i}}$, we propose the following algorithm for solving the problem:

1. Rank $\{i\}$ in order of $\frac{-a_{i}}{q_{i}}$ (lowest to highest). 
Note that under this sorting, for any $\tilde{v}$, if $\tilde{v} \leq \frac{-a_{j}}{q_{j}}$ for some $j$, then $\tilde{v} \leq \frac{-a_{k}}{q_{k}}$ for all $k > j$. 

2. For each $k$, let $\tilde{v}_{k}$ to be the value of $v$ that minimizes $(*)$ on the interval $(\frac{-a_{k}}{q_{k}}, \frac{-a_{k+1}}{q_{k+1}}]$. Do a line search over $k$ to find the minizer.

Note that for any $k$ in Step 2., there is a closed form solution to $\tilde{v}_{k}$:
$$
\tilde{v}_{k} = \arg\min\limits_{v \in \left(\frac{-a_{k}}{q_{k}}, \frac{-a_{k+1}}{q_{k+1}} \right] } \left\{ \left[\sum_{i \leq k} \frac{1}{4 b_{i}}\left(a_{i}+\tilde{v}_{k} q_{i}\right)^{2}\right] - \tilde{v}_{k} s\right\}.
$$

This is a sum of quadratics (e.g. a quadratic), and so we find the optimum by taking the FOC:
$$ \tilde{v}_{k}^{*} =
\min\left\{ \begin{array}{ll}
\frac{2 s - \sum\limits_{i \leq k} \frac{a_{i} q_{i}}{b_{i}}}{\sum\limits_{i \leq k} \frac{1}{b_{i}} q_{i}^{2}}\\
\end{array}, \frac{-a_{k+1}}{q_{k+1}} \right\}.
$$


### Making this algorith more robust

The algorithm above will work so long as $\frac{a_{i+} v q_{i}}{2 b_{i}}$ is well defined -- that is so long as $b_i > 0$. When there is (at least one) element $i$ such that $b_i = 0$ (and so it is linear), the optimal solution will stop propagating $v_k$'s as soon as it hits the first linear element in the $-a_i/q_i$ rank order. At that point (say the linear element is the $k$th one): $v_k = -a_k/q_k$ and $x_k = \frac{s - \sum\limits_{i \leq k} q_i x^{\ast}_{i}}{q_k}$.






## Julia Implementation

In [6]:
using JuMP
using Ipopt
using Plots

function quadprog_jumpSolve( a,b,q,s )
    # Compute the optimal unit bid vector for each score by solving the quadratic optimization problem

    m = Model(optimizer_with_attributes(Ipopt.Optimizer,  "print_level"=>0))
    
    @variables m begin
        x[i=1:length(a)] >= 0
    end
    
    @objective(m, Max, sum( (a[i]*x[i] - b[i]*(x[i])^2) for i=1:length(a) ))
    
    @constraint(m, sum( x[i] * q[i] for i=1:length(q) ) == s)
    
    optimize!(m)
    
    answer = objective_value(m)
    x_min = value.(x)
    
    return answer, x_min
    
end

quadprog_jumpSolve (generic function with 1 method)

In [73]:
mutable struct Bid
    init_index::Int64
    a::Float64
    b::Float64
    q::Float64
    a_over_q::Float64
end

function quadprog_exactSolve(a,b,q, s, detail = false)
    num_bids = length(a)
    bidvec = [Bid(i, a[i],b[i],q[i],(a[i]/q[i])) for i=1:num_bids ]
    sorted_bids = sort(bidvec, by = x -> x.a_over_q, rev=true)
    
    if detail
        println("sorted_bids: ")
        for bid in sorted_bids
            println(bid)
        end
        
    end
    
    ## Save sorted vector for faster calculations
    sorted_a = [sorted_bids[i].a for i=1:num_bids]
    sorted_b = [sorted_bids[i].b for i=1:num_bids]
    sorted_q = [sorted_bids[i].q for i=1:num_bids]
    min_sorted_a_over_q = [-sorted_bids[i].a_over_q for i=1:num_bids]
    min_sorted_a_over_q_nxt = [0.0 for i = 1:num_bids]
    
    for i=1:(num_bids-1)
        min_sorted_a_over_q_nxt[i] = min_sorted_a_over_q[i+1]
    end
    min_sorted_a_over_q_nxt[num_bids] = Inf


    ## Compute v_ks
    t1 = sorted_a .* sorted_q ./ sorted_b
    t2 = sorted_q.^2 ./ sorted_b
    t1_acc = accumulate(+, t1)
    t2_acc = accumulate(+, t2)
    
    ## Set v_ks and handle cases where b_i = 0
    xst_try = (2*s .- t1_acc) ./ t2_acc
    v_ks = copy(xst_try)
    v_ks = replace!(v_ks, NaN => Inf) 
    
    
    for i=1:(num_bids)
        v_ks[i] = sorted_b[i] <= 0 ? min_sorted_a_over_q[i] : min.(xst_try[i], min_sorted_a_over_q_nxt[i])
        if sorted_b[i] <= 0 ## only want to set the _first_ non-quad b
            break
        end
    end
    
    ## Compute duals
    function dual_eval(v)
        x_stars = [0.0 for  i=1:num_bids]
        dual_eval = 0
        cum_qx = 0
        for i=1:num_bids
            if min_sorted_a_over_q[i] > v
                break
            end
            if sorted_b[i] <= 0
                x_stars[i] = (s-cum_qx)/sorted_q[i] 
                dual_eval += ( (sorted_a[i]/sorted_q[i]) + v ) * (s - cum_qx)
                break
            else
                x_stars[i] = ( (sorted_a[i] + (v * sorted_q[i]) ) / (2 * sorted_b[i]) )
                dual_eval += ( (sorted_a[i] + (v * sorted_q[i]) )^2 / (4 * sorted_b[i]) )
                cum_qx +=  sorted_q[i]*x_stars[i]
            end
        end

        out = dual_eval - v*s
        return(out, x_stars)
    end
    
    duals = [dual_eval(v_k) for v_k in v_ks]
    
    
    if detail
        println("v_ks: ", v_ks)
        println("dual obj: ", [duals[i][1] for i=1:length(duals)])
    end
    
    sort!(duals, by = x -> x[1])
    
    opt = duals[1]
    opt_val = opt[1]
    x_star_sorted = opt[2]
    x_star_orig_order = copy(x_star_sorted)
    for i = 1:num_bids
        bid = sorted_bids[i]
        x_star_orig_order[bid.init_index] = x_star_sorted[i]
    end
    
    return(x_star_orig_order)
    
end


quadprog_exactSolve (generic function with 2 methods)

In [80]:
## A simple example
a = -[3,5,52]
b = -[0.,0.,0.4]
q = [1,3,5]
s = 35

println("JuMP Solution: ", quadprog_jumpSolve(a,b,q,s))

println("Our Solution: ", quadprog_exactSolve(a,b,q,s, true))

JuMP Solution: (-58.33333288834522, [0.0, 11.666666685944525, 0.0])
sorted_bids: 
Bid(2, -5.0, -0.0, 3.0, -1.6666666666666667)
Bid(1, -3.0, -0.0, 1.0, -3.0)
Bid(3, -52.0, -0.4, 5.0, -10.4)
v_ks: [1.6666666666666667, Inf, Inf]
dual obj: [-58.333333333333336, NaN, NaN]
Our Solution: [0.0, 11.666666666666666, 0.0]


## Putting this to Bids

In [88]:
function getOptimalBids(s, α, γ, W, c, p_bar, q_e ,sigma_sq)
    
    T = length(q_e)
    R = (W + s) - α*sum(c[t]*q_e[t] for t=1:T);
    R_sq = R^2;
    gm = γ - 1;
    
    
    b_tilde =  (R_sq*gm + 1)*(q_e .* q_e .* sigma_sq)
    a_tilde = (R * (q_e .* (2*p_bar + sigma_sq)) + (2 * α * c) .* b_tilde )
    s_tilde = s
    q_tilde = q_e
    
    sol = quadprog_exactSolve(a_tilde,b_tilde,q_tilde,s_tilde)
    
    return(sol)
end

getOptimalBids (generic function with 2 methods)

In [89]:
s0 = 30
α0 = 2.5
γ0 = 0.5
W0 = 0.0

qa0 = [12; 40];
qe0 = [10; 50];
c0 = [5;10];
sigmasq0 = [0.05,0.02];

T0 = length(c0)
pbar0 = [log(qa0[t]/qe0[t]) for t = 1:T0]

getOptimalBids(s0, α0, γ0, W0, c0, pbar0, qe0 ,sigmasq0)

2-element Array{Float64,1}:
 3.0
 0.0

In [90]:
b_opt_of_s = x -> getOptimalBids(x,  α0, γ0, W0, c0, pbar0, qe0 ,sigmasq0)
b_opt_of_s(s0)

2-element Array{Float64,1}:
 3.0
 0.0

In [92]:
using ForwardDiff
db_ds = ForwardDiff.derivative(b_opt_of_s, s0)

MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#446#447",Int64},Float64,1})
Closest candidates are:
  Float64(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  Float64(::T) where T<:Number at boot.jl:718
  Float64(!Matched::Int8) at float.jl:60
  ...

## Adding Constraints

Suppose that we add item-specific constraints, so that our problem is:

$$
\max\limits_{\{x_{i}\}_i} \sum_{i} a_{i} x_{i} - b_{i} x_{i}^{2} \\
\text{subject to }
\left\{\begin{array}{l}
        \sum_{i} q_{i} x_{i} = s  \\
%         \text{ and } \\
        x_{i} \geq r_i  \text{ for each } i 
    \end{array}\right.
$$

where $r_i > 0$ is some (known) number for each component $i$.

To use our algorithm above, we simply transform $x$ into a new variable: $y = x - r$

$$
\max\limits_{\{y_{i}\}_i} \sum_{i} a_{i} (y_{i} + r_{i}) - b_{i} (y_{i} + r_{i})^{2} \\
\text{subject to }
\left\{\begin{array}{l}
        \sum_{i} q_{i} (y_{i} + r_{i}) = s  \\
%         \text{ and } \\
        y_{i} \geq 0  \text{ for each } i 
    \end{array}\right.
$$

Simplifying, we see that this fits right into our previous framework:

$$
\max\limits_{\{y_{i}\}_i} \sum_{i} \tilde{a}_{i} y_{i} - \tilde{b}_{i} y_{i}^{2} + \tilde{C}_{i} \\
\text{subject to }
\left\{\begin{array}{l}
        \sum_{i} q_{i} y_{i} = \tilde{s}  \\
        y_{i} \geq 0  \text{ for each } i 
    \end{array}\right.\\
%     
    \text{where: }
\left\{\begin{array}{l}
        \tilde{a}_{i} = a_{i} - 2 b_{i} r_{i}  \\
        \tilde{b}_{i} = b_{i} \\
        \tilde{C}_{i} = a_{i} r_{i} - b_{i} r_{i}^2 \\
        \tilde{s} = s - \sum_{i}{q_{i} r_{i}}
    \end{array}\right.
$$

Note that $\tilde{C}$ is a constant and so does not affect optimization.


## Julia Implementation

In [17]:
function quadprog_jumpSolve_constraints( a,b,q,s,r )
    # Compute the optimal unit bid vector for each score by solving the quadratic optimization problem

    m = Model(optimizer_with_attributes(Ipopt.Optimizer,  "print_level"=>0))
    
    @variables m begin
        x[i=1:length(a)] >= r[i]
    end
    
    @objective(m, Max, sum( (a[i]*x[i] - b[i]*(x[i])^2) for i=1:veclen ))
    
    @constraint(m, sum( x[i] * q[i] for i=1:length(q) ) == s)
    
    optimize!(m)
    
    answer = objective_value(m)
    x_min = value.(x)
    
    return answer, x_min
    
end

quadprog_jumpSolve_constraints (generic function with 1 method)

In [51]:
function quadprog_fastSolve_constraints(a,b,q,s,r, detail = false)
    a_tilde =  a - 2*b.*r
    b_tilde = b
    s_tilde = s - sum(q[i]*r[i] for i=1:length(a))
    
    transformed_sol = quadprog_fastSolve(a_tilde,b_tilde,q,s_tilde)
    transformed_obj =  transformed_sol[1]
    raw_obj = transformed_obj + sum(a[i]*r[i] - b[i]*(r[i]^2) for i=1:length(a)) 
    
    y = transformed_sol[2]
    x = y + r
    return(raw_obj, x)
end

quadprog_fastSolve_constraints (generic function with 2 methods)

In [59]:
## A simple example
a = -[3,50,52]
b = [0.5,14,2]
q = [1,3,5]
s = 50
r = [2,2,2]

veclen = length(a)

println("JuMP Solution: ", quadprog_jumpSolve_constraints(a,b,q,s,r))

println("Our Solution: ", quadprog_fastSolve_constraints(a,b,q,s,r ))

JuMP Solution: (-678.6034470852646, [12.448275870433951, 2.0, 6.310344837888006])
Our Solution: (-678.6034482758621, [12.448275862068966, 2.0, 6.310344827586206])
